In [1]:
%load_ext autoreload
%autoreload 2

import os
from alphadia.outputtransform import QuantBuilder
from alphabase.spectral_library import base
from alphadia import outputtransform
import pandas as pd

2025-03-11 12:59:36,894 - rdkit - INFO - Enabling RDKit 2024.03.3 jupyter extensions
/Users/tungvuduc/opt/anaconda3/envs/alpha_dia/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
base_spec_lib = base.SpecLibBase()
quant_folder = '/Users/tungvuduc/Desktop/PhD/projects/repos/selectLFQ/selectlfq/data/benchmark_data/MixedSpeciesOlsen/alphadia_1.10.0/second_pass/quant'

folder_list = [os.path.join(quant_folder, f) for f in os.listdir(quant_folder) if os.path.isdir(os.path.join(quant_folder, f))]
output_folder = '/Users/tungvuduc/Desktop/PhD/projects/repos/selectLFQ/selectlfq/data/benchmark_data/MixedSpeciesOlsen/alphadia_1.10.0/second_pass'

In [11]:
config = {
        "general": {
            "thread_count": 8,
        },
        "fdr": {
            "fdr": 0.01,
            "inference_strategy": 'maximum_parsimony',
            "group_level": "genes",
            "keep_decoys": False,
        },
        "search_output": {
            "min_k_fragments": 12,
            "min_correlation": 0.9,
            "num_samples_quadratic": 50,
            "min_nonnan": 3,
            "peptide_level_lfq": False,
            "precursor_level_lfq": True,
            "normalize_lfq": True,
            "file_format": "tsv",
        },
    }

output = outputtransform.SearchPlanOutput(config, output_folder)


In [12]:
output = outputtransform.SearchPlanOutput(config, output_folder)
psm_df = output.load_precursor_table()

2025-03-11 15:56:53> Reading /Users/tungvuduc/Desktop/PhD/projects/repos/selectLFQ/selectlfq/data/benchmark_data/MixedSpeciesOlsen/alphadia_1.10.0/second_pass/precursors.tsv from disk


In [13]:
qb = QuantBuilder(psm_df[psm_df["decoy"] == 0])
df_list = qb.accumulate_frag_df_from_folders(folder_list)

2025-03-11 15:57:18> Accumulating fragment data
2025-03-11 15:57:18> reading frag file for 20230324_OLEP08_200ng_30min_E40H50Y10_180K_2Th3p5ms_03
2025-03-11 15:57:18> reading frag file for 20230324_OLEP08_200ng_30min_E10H50Y40_180K_2Th3p5ms_01
2025-03-11 15:57:31> reading frag file for 20230324_OLEP08_200ng_30min_E40H50Y10_180K_2Th3p5ms_02
2025-03-11 15:57:41> reading frag file for 20230324_OLEP08_200ng_30min_E30H50Y20_180K_2Th3p5ms_02
2025-03-11 15:57:50> reading frag file for 20230324_OLEP08_200ng_30min_E45H50Y5_180K_2Th3p5ms_03
2025-03-11 15:58:00> reading frag file for 20230324_OLEP08_200ng_30min_E20H50Y30_180K_2Th3p5ms_01
2025-03-11 15:58:10> reading frag file for 20230324_OLEP08_200ng_30min_E30H50Y20_180K_2Th3p5ms_03
2025-03-11 15:58:20> reading frag file for 20230324_OLEP08_200ng_30min_E45H50Y5_180K_2Th3p5ms_02
2025-03-11 15:58:30> reading frag file for 20230324_OLEP08_200ng_30min_E5H50Y45_180K_2Th3p5ms_01
2025-03-11 15:58:40> reading frag file for 20230324_OLEP08_200ng_30min_E1

In [21]:
# save the feature data
# create a feature folder in the output folder
feature_folder = os.path.join(output_folder, "features")
if not os.path.exists(feature_folder):
    os.makedirs(feature_folder, exist_ok=True)

for feature, df in zip(qb.column, df_list):
    df.to_csv(f'{feature_folder}/{feature}.csv')